In [10]:
import json
import pennylane as qml
import pennylane.numpy as np

# Write any helper functions you need
def S(phi):
    return np.array([
        [np.exp(1j * phi), 0],
        [0, np.exp(-1j * phi)]
    ])

def U(a):
    b = np.sqrt(1 - a**2)
    return np.array([
        [a, b],
        [b, -a]
    ])

def coefficients(angles):
    
    """ This function returns the coefficients associated with the polynomial generated by 
    the QSP routine as a function of the phase angles.
    
    Args:
        - angles (np.array(float)): Array of real numbers containing the four phase angles,
        in reverse order of application.
    
    Returns:       
        - (np.array(complex)): A numpy array containing the coefficients of the polynomial
        generated by QSP, where the first element is the coefficient of the cubic term and
        the second element is for the linear term.
    """
    


    # Put your code here
    def P(a):
        circuit_U = S(angles[3]) @ U(a) @ S(angles[2]) @ U(a) @ S(angles[1]) @ U(a) @ S(angles[0])
        return circuit_U[0][0]

    alpha = (P(1) - 2*P(0.5)) * (4 / 3)
    beta = (P(1) - 8*P(0.5)) / (-3)
    
    return np.array([alpha, beta])

# These functions are responsible for testing the solution.


def run(test_case_input: str) -> str:
    ins = np.array(json.loads(test_case_input))
    coeffs = coefficients(ins)
    outs = [[np.real(elem).numpy(), np.imag(elem).numpy()] for elem in coeffs]
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, rtol = 1e-4)


# These are the public test cases
test_cases = [
    ('[0.5,0.8,1.0,1.0]', '[[-0.1707976, -2.4084889], [-0.8166822, 2.2507432]]'),
    ('[-0.20409113, -0.91173829, 0.91173829, 0.20409113]', '[[2.5,0],[-1.5,0]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.5,0.8,1.0,1.0]'...
Correct!
Running test case 1 with input '[-0.20409113, -0.91173829, 0.91173829, 0.20409113]'...
Correct!


In [6]:
def coefficients(angles):
    # Define o número de ângulos
    num_angles = len(angles)

    # Inicializa a matriz associada aos ângulos de fase
    phase_matrix = np.identity(2)

    # Loop reverso sobre os ângulos
    for angle in reversed(angles):
        phase_matrix = np.dot(qml.S(angle), phase_matrix)

    # Define a função de transformação do sinal
    def U(a):
        return np.array([[a, 1-a], [1-a, -a]])

    # Calcula a matriz associada ao circuito QSP
    qsp_matrix = np.identity(2)
    for _ in range(num_angles):
        qsp_matrix = np.dot(U(0), np.dot(phase_matrix, qsp_matrix))

    # Extrai os coeficientes do polinômio
    alpha = qsp_matrix[1, 0]
    beta = qsp_matrix[0, 0]

    return np.array([alpha, beta])

In [7]:
angles = [0.5,0.8,1.0,1.0]
coefficients(angles)

TypeError: unsupported operand type(s) for *: 'S' and 'SProd'